In [ ]:
Crawler + Scrapper
HTTP - Req/Resp - Bytes - (TEXT/HTML)String => Requests, BeautifulSoup
HTML -> DOM : find~, select(CSS Selector)
HTML Tag, #id, .class, HTML Attributes, :가상선택자, [키(^$+)=밸류]
Crawler; HyperLink Web 탐색(a[href], iframe[src], img, ...)
    -> scheme://netloc(server domain)/path?params#fragment(X)
    BFS(Queue), DFS(Stack) : 검색결과 ; Depth
Link Analysis => PageRank, 스크래핑

In [3]:
from urllib.robotparser import RobotFileParser
from requests import get
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
from time import sleep
import re
from bs4 import BeautifulSoup


headers = {
    'user-agent':'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}
URLs = list()
seens = list()
URLs.append({'url' : 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%ED%95%9C%EC%86%8C%ED%9D%AC',
            'depth': 0
}) # 구조 변경. 기존 list에서 dict의 list로(key:url, depth)

# 전략1. depth(0 -> 1 -> 2, ...)
#        [{url: 'url', deprh: 0}, ...]
# 전략2. domain
#        blog.naver.com
allowedDomain = ['blog.naver.com', 'postfiles.pstatic.net']

while URLs:
    seed = URLs.pop(0) # BFS:0, DFS:-1
    seens.append(seed['url'])
 
    # 전략 1 적용
    if seed['depth'] > 3:
        continue
        
    # list에서 꺼낸 url은 dict 이므로, 실제주소는 dict의 key:url
    resp = get(seed['url'], headers=headers)

    try:
        resp.raise_for_status()
    except HTTPError as e:
        print(e)
        continue

    # 전랴3: 텍스트/html + image/format
    if re.search('text|html|image|jpeg|png|gif|bmp', resp.headers['content-type']) is None:
        continue
    
    if re.search('image|jpeg|png|gif|bmp', resp.headers['content-type']):
        # https://blog.naver.com/path1/path2/(image12432@!~14123)
        filename = resp.url.split('/')[-1]
        # (image12432@!~14123) => (image1243214123)
        filename = re.sub('[?#!]', '', filename)
        # image/(___)
        ext = re.search('image/(\w+);?',
                       resp.headers['content-type']).group(1)
        #filename: ./data/imgae1243214123.ext
        with open('./data/'+filename+'.'+ext, 'wb') as fp:
            fp.write(resp.content)
    else:
        # 위와 동일하게, filename 만들어서 w모드 저장(encoding) resp.text
        dom = BeautifulSoup(resp.text, 'html.parser')
        
        for link in dom.select('a[href], iframe[src], img[src]'):
            url = urljoin(seed['url'], link.attrs['href']
                          if link.has_attr('href') else link.attrs['src'])

            if len(urlparse(url).fragment) == 0 and\
                urlparse(url).scheme in ['http', 'https']:
                # {depth제한} => list의 dict 풀어서 => 주소만 있는 list
                # 전략2 적용 blog.naver.com
                if url not in [u['url'] for u in URLs] and \
                    url not in seens and \
                    urlparse(url).netloc in allowedDomain:
                    # 앞으로 방문할 URL목록에 dict로 추가
                    URLs.append({'url':url, 'depth':seed['depth']+1})
        print(len(URLs))

16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
72
99
133
160
189
218
252
267
310
343
356
389
411
431
457
460
459
458
458
461
460
473
478


OSError: [Errno 22] Invalid argument: './data/2021-03-02_10:19:55.631.jpgtype=w80_blur.jpeg'

In [ ]:
# 웹툰 크롤링 + 스크래핑
# 특정 도메인, 특정 영역, Depth X
URLs = ['https://comic.naver.com/webtoon']
visited = list()

while URLs:
    seed = URLs